### Objective
This notebook aims to:
  - check quickly the stationarity of the sales series
  - Preprocess (basic) the dataset including new features creation

In [1]:
import sys
import os
import missingno as msno
import logging
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

sns.set_style("darkgrid")
%matplotlib inline

import warnings
warnings.filterwarnings("ignore")

sys.path.append(os.path.abspath(os.path.join("../scripts"))) 
import load_data

# from data_infos import dataframeInfo
from clean import dataCleaning

from statsmodels.tsa.stattools import adfuller

In [2]:
logging.basicConfig(filename='../exploration_logfile.log', filemode='a',
                    encoding='utf-8', level=logging.DEBUG)

## Load data

In [3]:
# data = load_data.load_data('../data/train_data.csv')
# data = dataCleaning(data).cleanStateHoliday()
# data = dataCleaning(data).dateformat('Date')

In [4]:
import dvc.api
import io

path = 'data/train_data.csv'
repo = './'
version='v1.2'

data_url = dvc.api.get_url(
    path=path,
	repo=repo,
	rev=version
	)

data = pd.read_csv(data_url, sep=",")
data = dataCleaning(data).cleanStateHoliday()
data = dataCleaning(data).dateformat('Date')

In [5]:
import dvc.api
import io

path = 'data/test_data.csv'
repo = './'
version='vt.1'

data_url = dvc.api.get_url(
    path=path,
	repo=repo,
	rev=version
	)

test = pd.read_csv(data_url, sep=",")
test = dataCleaning(test).cleanStateHoliday()

In [6]:
data.head()

,Date,Store,DayOfWeek,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,Year,...,WeekOfYear,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
0,2015-07-31,1,5,5263,555,1,1,0,1,2015,...,31,c,a,1270.0,9.0,2008.0,0,0.0,0.0,0
1,2015-07-30,1,4,5020,546,1,1,0,1,2015,...,31,c,a,1270.0,9.0,2008.0,0,0.0,0.0,0
2,2015-07-29,1,3,4782,523,1,1,0,1,2015,...,31,c,a,1270.0,9.0,2008.0,0,0.0,0.0,0
3,2015-07-28,1,2,5011,560,1,1,0,1,2015,...,31,c,a,1270.0,9.0,2008.0,0,0.0,0.0,0
4,2015-07-27,1,1,6102,612,1,1,0,1,2015,...,31,c,a,1270.0,9.0,2008.0,0,0.0,0.0,0


In [7]:
test.head()

,Id,Store,DayOfWeek,Open,Promo,StateHoliday,SchoolHoliday,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
0,1,1,4,1.0,1,0,0,c,a,1270.0,9.0,2008.0,0,0.0,0.0,0
1,857,1,3,1.0,1,0,0,c,a,1270.0,9.0,2008.0,0,0.0,0.0,0
2,1713,1,2,1.0,1,0,0,c,a,1270.0,9.0,2008.0,0,0.0,0.0,0
3,2569,1,1,1.0,1,0,0,c,a,1270.0,9.0,2008.0,0,0.0,0.0,0
4,4281,1,6,1.0,0,0,0,c,a,1270.0,9.0,2008.0,0,0.0,0.0,0


## Quick stationarity checking

In [8]:
data.set_index('Date', inplace=True)

In [9]:
X = data['Sales'].values
split = round(len(X) / 2)
X1, X2 = X[0:split], X[split:]
mean1, mean2 = X1.mean(), X2.mean()
var1, var2 = X1.var(), X2.var()
print('mean1=%f, mean2=%f' % (mean1, mean2))
print('variance1=%f, variance2=%f' % (var1, var2))

mean1=5801.525100, mean2=5746.112899
variance1=14744172.399597, variance2=14898126.160203


### The two means are approximatively the same: one can consider the sales as a stationary series.

#### Let's sort the day from 2013 to 2015

In [10]:
data = data.sort_index()

### Data preprocessing

It is important to process the data into a format where it can be fed to a machine learning model. This typically means converting all non-numeric columns to numeric, handling NaN values and generating new features from already existing features. 

In our case, we have a few datetime columns to preprocess.  
We will extract the following from them:
- weekdays
- weekends 
- Beginning of month, mid month and ending of month
- etc.

At this stage, the current version of the data is already cleaned. We will straight to the processing.

### New features creation

- We consider that Monday to Friday are weekdays while Saturday and Sunday are weekends.  
- A day is holiday if match with either one of states holidays (0 if holiday and 1 if not)
- We consider that the begining of the month is within the first 7 days
- We consider that the end of the month is within the last 7 days
- We consider that the mid of the month is between the 10th and the 20th day

Note that **Month, Year, Day and WeekofYear** has been created during the exploration. Then, they already exist in this version of the data.

In [11]:
class TestaddTransform():
    ###############################################################################
    # New feature and basic transformation
    ################################################################################

    def __init__(self, df: pd.DataFrame):
        """
            Returns a dataframe Info Object with the passed DataFrame Data
            Parameters
        """
        self.df = df

    def testnewfeatures(self):
        self.df['WeekDays'] = (self.df['DayOfWeek'] <= 5)*1
        self.df['WeekEnds'] = (self.df['DayOfWeek'] > 5)*1
        self.df['IsHoliday'] = self.df['StateHoliday'].map({0: 0, "a": 1, "b": 1, "c": 1})
        self.df['IsHoliday'] = self.df['IsHoliday'].astype(int) 
        self.df['StateHoliday'] = self.df['StateHoliday'].map({0: "0", "a": "a", "b": "b", "c": "c"})
        self.df['PromoPerCompetitionDistance'] = self.df.groupby('Promo')['CompetitionDistance'].transform('mean')
        self.df['Promo2PerCompetitionDistance'] = self.df.groupby('Promo2SinceWeek')['CompetitionDistance'].transform('mean')

        return self.df

    def dropcol(self):
        ### Drop non-relevant columns
        self.df.drop(['Open', 'Store'], axis=1, inplace=True)

        return self.df  
    
    def dropcoltest(self):
        self.df.drop(['Id'], axis=1, inplace=True)
        return self.df
    
    def removeClosedStores(self):
        """
            Closed stores and days which didn't have any sales won't be counted into the forecasts.
        """

        self.df = self.df[self.df['Open'] == 1]
        return self.df

In [12]:
class TrainaddTransform(TestaddTransform):
    ###############################################################################
    # New feature and basic transformation
    ################################################################################

    def __init__(self, df: pd.DataFrame):
        """
            Returns a dataframe Info Object with the passed DataFrame Data
            Parameters
        """
        self.df = df

    def newfeatures(self): 
        self.df['BeginMonth'] = (((self.df['Day'])//7) == 0)*1
        self.df['MidMonth'] = (((self.df['Day'])//10) == 1)*1
        self.df['EndMonth'] = (((self.df['Day'])//7) >= 3)*1
        

        return self.df

    def dropcoltrain(self):
        ### Drop non-relevant columns
        self.df.drop(['Customers'], axis=1, inplace=True)

        return self.df  

In [13]:
# remove closed stores and those with no sales
train = dataCleaning(data).removeClosedStores()
test = TestaddTransform(test).removeClosedStores()

In [14]:
train = TrainaddTransform(train).testnewfeatures()
train = TrainaddTransform(train).newfeatures()
train = TrainaddTransform(train).dropcol()
train = TrainaddTransform(train).dropcoltrain()

test = TestaddTransform(test).testnewfeatures()
test = TestaddTransform(test).dropcol()
test = TestaddTransform(test).dropcoltest()

In [15]:
train.reset_index(inplace=True)

In [16]:
train.head()

,Date,DayOfWeek,Sales,Promo,StateHoliday,SchoolHoliday,Year,Month,Day,WeekOfYear,...,Promo2SinceYear,PromoInterval,WeekDays,WeekEnds,IsHoliday,PromoPerCompetitionDistance,Promo2PerCompetitionDistance,BeginMonth,MidMonth,EndMonth
0,2013-01-01,2,3375,0,a,1,2013,1,1,1,...,0.0,0,1,0,1,5466.797276,6549.550665,1,0,0
1,2013-01-01,2,2907,0,a,1,2013,1,1,1,...,0.0,0,1,0,1,5466.797276,6549.550665,1,0,0
2,2013-01-01,2,6851,0,a,1,2013,1,1,1,...,0.0,0,1,0,1,5466.797276,6549.550665,1,0,0
3,2013-01-01,2,17267,0,a,1,2013,1,1,1,...,0.0,0,1,0,1,5466.797276,6549.550665,1,0,0
4,2013-01-01,2,3102,0,a,1,2013,1,1,1,...,2013.0,"Jan,Apr,Jul,Oct",1,0,1,5466.797276,5048.186871,1,0,0


In [17]:
test.head()

,DayOfWeek,Promo,StateHoliday,SchoolHoliday,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval,WeekDays,WeekEnds,IsHoliday,PromoPerCompetitionDistance,Promo2PerCompetitionDistance
0,4,1,0,0,c,a,1270.0,9.0,2008.0,0,0.0,0.0,0,1,0,0,5085.789020,6727.628859
1,3,1,0,0,c,a,1270.0,9.0,2008.0,0,0.0,0.0,0,1,0,0,5085.789020,6727.628859
2,2,1,0,0,c,a,1270.0,9.0,2008.0,0,0.0,0.0,0,1,0,0,5085.789020,6727.628859
3,1,1,0,0,c,a,1270.0,9.0,2008.0,0,0.0,0.0,0,1,0,0,5085.789020,6727.628859
4,6,0,0,0,c,a,1270.0,9.0,2008.0,0,0.0,0.0,0,0,1,0,5189.642611,6727.628859


In [18]:
train.to_csv('../data/good_train.csv', index=False)

In [19]:
test.to_csv('../data/good_test.csv', index=False)